In [2]:
#Dependencies
import requests 
import time
from joblib import Parallel, delayed

In [3]:
#Search for specific data in RNA central
url = 'https://rnacentral.org/api/v1/rna'
response = requests.get(url, params = {
    "max_length":500 , "min_length":10 , "tax_string":"Bacteria", "so_rna_type_name":"Small_regulatory_ncRNA" ,
    "page_size":100 
})
assert(response.status_code==200)

In [4]:
#Extract information from json entry
def extract_info(entry):
    result=[]    
    #TODO extract species info
    result.append("idk")
    result.append(entry["sequence"])        
    result.append("RNAcentral")
    #TODO extract function info
    result.append("idk")
    return result

In [11]:
#Iterate through query result 
#Create | Tax_info / Sequence / Source / Function | list of lists
max_retry=5
wait=60
result=[]
counter=0
#Do-while loop; no idea how long it takes since count is bugged form the API. Subect to empirical testing
while True:
        print("counter:"+str(counter))
        counter=counter+1
        response_json=response.json()
        #go concurrent or go home
        tmp=Parallel(n_jobs=snakemake.threads)(delayed(extract_info)(i) for i in response_json["results"])
        result=result+tmp
        print(result)
        next_response=response_json["next"]
        if next_response is None:
            break
        #try to get next response
        tries=0
        response=requests.get(next_response)
        while (response.status_code!=200 and tries<max_retry):
            print("tries:"+str(tries))
            time.sleep(wait)
            response=requests.get(next_response)
            tries= tries+1
        assert(response.status_code==200)


In [5]:
response.json()

In [ ]:
 with open(snakemake.output[0], 'w', newline="") as s
        csv.writer(s,delimiter="\t").writerows(result)